<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fixed" data-toc-modified-id="Fixed-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Fixed</a></span></li></ul></div>

In [27]:
import pandas as pd
import time
import numpy as np
import openai
from tqdm import tqdm
import pickle
import re
import sys

# OpenAI API Key
openai.api_key = "PUT YOUR API KEY HERE"

# Query function
def do_query(prompt, max_tokens=2):
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        temperature=0.7,
        max_tokens=max_tokens,
        top_p=1,
        logprobs=100,
    )
    time.sleep(1.0)  # to avoid rate limiters
    return response

# Function to get unique values
def uniqvals(users, field):
    vals = [users[id][field] for id in users.keys()]
    return list(set(vals))

# Fields of interest
fields_of_interest = {
    "Gender": {"Male": "male", "Female": "female", '': ''},
    "Hisp": {"Hispanic": "Hispanic", "Not Hispanic": '', '': ''},
    "WHITE": {"White": "white", "Non-white": "", '': ''},
    "Ideo": {
        '': '',
        'Liberal': 'liberal',
        'Slightly conservative': 'slightly conservative',
        'Conservative': 'conservative',
        'Slightly liberal': 'slightly liberal',
        "Moderate/Haven't thought about it": 'moderate',
        'Extremely Liberal': 'extremely liberal',
        'Extremely conservative': 'extremely conservative',
    },
    "PID7": {
        '': '',
        'Ind': 'am an independent',
        'Strong D': 'am a strong Democrat',
        'Strong R': 'am a strong Republican',
        'Lean D': 'lean towards Democrats',
        'Lean R': 'lean towards Republicans',
        'Weak D': 'am a weak Democrat',
        'Weak R': 'am a weak Republican',
    },
    "Inc": {
        '': '',
        'Less than $15K': 'very poor',
        '$15K to $25K': 'poor',
        '$25K to $50K': 'poor',
        '$50K to $75K': 'middle-class',
        '$75K to $100K': 'middle-class',
        '$100K to $150K': 'middle-class',
        '$150K to $200K': 'upper-class',
        '$200K to $250K': 'upper-class',
        '$250K to $500K': 'upper-class',
        'Prefer not to answer': '',
        '-8': '',
    },
}


def mapper(profile):
    results = {}
    for k in profile.keys():
        if k in fields_of_interest:
            results[k] = fields_of_interest[k][profile[k]]
    if profile['Age'] != '':
        age = int(profile['Age'])
        if age >= 18 and age < 25:
            results['Age'] = 'young'
        if age >= 25 and age < 40:
            results['Age'] = 'middle-aged'
        if age >= 40 and age < 60:
            results['Age'] = 'old'
        if age >= 60 and age < 100:
            results['Age'] = 'very old'
    else:
        results['Age'] = ''
    return results

# Read data from file
lines = open("../out_of_many/main/Study1_data/ppfull.csv", "r").readlines()
dmap = {}
header_vals = lines[0].strip().split(",")
for line in lines[1:]:
    parts = line.strip().split(",")
    if len(parts) != len(header_vals):
        print("Error on line: " + line)
        continue
    dmap[parts[0]] = {}
    for hind, h in enumerate(header_vals):
        dmap[parts[0]][header_vals[hind]] = parts[hind]

# List of parties
party_list = ["Republican", "Democratic"]


# Results dictionary
results = {}
ids = dmap.keys()
for id in tqdm(ids):
    results[id] = {}
    user_profile = mapper(dmap[id])
    print(user_profile.keys())
    for party in party_list:
        print(f"---------------------------------------- {id} {party}")
        prompt = ""
        if user_profile['Ideo'] != '':
            prompt += "Ideologically, I describe myself as " + user_profile['Ideo'] + ". "
        if user_profile['PID7'] != '':
            prompt += "Politically, I " + user_profile['PID7'] + ". "
        if user_profile['WHITE'] == 'White':
            prompt += "Racially, I am white. "
        if user_profile['Hisp'] == 'Hispanic':
            prompt += "Racially, I am Hispanic. "
        if user_profile['Gender'] != '':
            prompt += "I am " + user_profile['Gender'] + ". "
        if user_profile['Inc'] != '':
            prompt += "Financially, I am " + user_profile['Inc'] + ". "
        if user_profile['Age'] != '':
            prompt += "In terms of my age, I am " + user_profile['Age'] + ". "
        prompt += "When I am asked to write down four words that typically describe people"
        prompt += " who support the " + party + " Party, I respond with: 1."
        print(prompt)
#         full_response = do_query(prompt, max_tokens=128)
#         text = full_response['choices'][0]['text']
#         text = "1." + text  # since "1." is part of the prompt
#         print(text)
#         results[id][party] = text

# Save results to pickle file


Error on line: R_5cf9ug7blHKzEDX,"$500K to $1,000K",Not Hispanic,Liberal,38,liberal,socially conscious,politically correct,kind,wealthy,conservative,closed minded,ignorant,Weak D,Democrat,White,4,4,Female,wealthy|conservative|closed minded|ignorant,liberal|socially conscious|politically correct|kind,0,0

Error on line: R_eb4thrlwqSPNGi9,"More than $1,000K",Hispanic,Moderate/Haven't thought about it,24,poor,,,,rich,,,,Strong R,Republicans,Non-white,1,1,Male,rich|||,poor|||,1,1

Error on line: R_ebxaYTTERHs3RJP,"More than $1,000K",Not Hispanic,Conservative,26,poor,black,lazy,uneducated,smart,powerful,values,educated,Strong R,Republicans,White,4,4,Female,smart|powerful|values|educated,poor|black|lazy|uneducated,0,0

Error on line: R_D7RQwYPEczU0vQt,$25K to $50K,Not Hispanic,Liberal,25,,"left wing, people minded, considerate, pro-choice",,,"stubborn, backwards, racist, rich",,,,Weak D,Democrat,White,1,1,Female,"stubborn, backwards, racist, rich|||","|left wing, people minded, considerate, 

 13%|█▎        | 280/2107 [00:00<00:00, 2444.65it/s]

dict_keys(['Inc', 'Hisp', 'Ideo', 'PID7', 'WHITE', 'Gender', 'Age'])
---------------------------------------- R_dbTpgNCUaQtIifr Republican
Ideologically, I describe myself as slightly conservative. Politically, I am an independent. I am female. Financially, I am middle-class. In terms of my age, I am old. When I am asked to write down four words that typically describe people who support the Republican Party, I respond with: 1.
---------------------------------------- R_dbTpgNCUaQtIifr Democratic
Ideologically, I describe myself as slightly conservative. Politically, I am an independent. I am female. Financially, I am middle-class. In terms of my age, I am old. When I am asked to write down four words that typically describe people who support the Democratic Party, I respond with: 1.
dict_keys(['Inc', 'Hisp', 'Ideo', 'PID7', 'WHITE', 'Gender', 'Age'])
---------------------------------------- R_51QVYKmeIc9iLpH Republican
Ideologically, I describe myself as slightly conservative. Politic

dict_keys(['Inc', 'Hisp', 'Ideo', 'PID7', 'WHITE', 'Gender', 'Age'])
---------------------------------------- R_a33Z2gdui5hMUh7 Republican
Ideologically, I describe myself as conservative. Politically, I am a weak Republican. I am male. Financially, I am middle-class. In terms of my age, I am old. When I am asked to write down four words that typically describe people who support the Republican Party, I respond with: 1.
---------------------------------------- R_a33Z2gdui5hMUh7 Democratic
Ideologically, I describe myself as conservative. Politically, I am a weak Republican. I am male. Financially, I am middle-class. In terms of my age, I am old. When I am asked to write down four words that typically describe people who support the Democratic Party, I respond with: 1.
dict_keys(['Inc', 'Hisp', 'Ideo', 'PID7', 'WHITE', 'Gender', 'Age'])
---------------------------------------- R_bOYts3BV1yLWVRr Republican
Ideologically, I describe myself as moderate. Politically, I lean towards Democrat

 34%|███▍      | 715/2107 [00:00<00:00, 2295.85it/s]


dict_keys(['Inc', 'Hisp', 'Ideo', 'PID7', 'WHITE', 'Gender', 'Age'])
---------------------------------------- R_4MyHODIz8AaIfNr Republican
Ideologically, I describe myself as slightly liberal. Politically, I am a strong Democrat. Racially, I am Hispanic. I am female. Financially, I am middle-class. In terms of my age, I am middle-aged. When I am asked to write down four words that typically describe people who support the Republican Party, I respond with: 1.
---------------------------------------- R_4MyHODIz8AaIfNr Democratic
Ideologically, I describe myself as slightly liberal. Politically, I am a strong Democrat. Racially, I am Hispanic. I am female. Financially, I am middle-class. In terms of my age, I am middle-aged. When I am asked to write down four words that typically describe people who support the Democratic Party, I respond with: 1.
dict_keys(['Inc', 'Hisp', 'Ideo', 'PID7', 'WHITE', 'Gender', 'Age'])
---------------------------------------- R_1S8aLCA4KIvUM6h Republican
Ideo

KeyError: 'Age'

# Fixed

Skip rows that throw an error. Don't add the GPT generated words for partisans. 

In [34]:
import pandas as pd
import time
import numpy as np
import openai
from tqdm import tqdm
import pickle
import re
import sys

# OpenAI API Key
openai.api_key = "PUT YOUR API KEY HERE"

# Fields of interest
fields_of_interest = {
    "Gender": {"Male": "male", "Female": "female", '': ''},
    "Hisp": {"Hispanic": "Hispanic", "Not Hispanic": '', '': ''},
    "WHITE": {"White": "white", "Non-white": "", '': ''},
    "Ideo": {
        '': '',
        'Liberal': 'liberal',
        'Slightly conservative': 'slightly conservative',
        'Conservative': 'conservative',
        'Slightly liberal': 'slightly liberal',
        "Moderate/Haven't thought about it": 'moderate',
        'Extremely Liberal': 'extremely liberal',
        'Extremely conservative': 'extremely conservative',
    },
    "PID7": {
        '': '',
        'Ind': 'am an independent',
        'Strong D': 'am a strong Democrat',
        'Strong R': 'am a strong Republican',
        'Lean D': 'lean towards Democrats',
        'Lean R': 'lean towards Republicans',
        'Weak D': 'am a weak Democrat',
        'Weak R': 'am a weak Republican',
    },
    "Inc": {
        '': '',
        'Less than $15K': 'very poor',
        '$15K to $25K': 'poor',
        '$25K to $50K': 'poor',
        '$50K to $75K': 'middle-class',
        '$75K to $100K': 'middle-class',
        '$100K to $150K': 'middle-class',
        '$150K to $200K': 'upper-class',
        '$200K to $250K': 'upper-class',
        '$250K to $500K': 'upper-class',
        'Prefer not to answer': '',
        '-8': '',
    },
}



# Query function
def do_query(prompt, max_tokens=2):
    response = openai.Completion.create(
        engine="davinci",
        prompt=prompt,
        temperature=0.7,
        max_tokens=max_tokens,
        top_p=1,
        logprobs=100,
    )
    time.sleep(1.0)  # to avoid rate limiters
    return response

# Function to get unique values
def uniqvals(users, field):
    vals = [users[id][field] for id in users.keys()]
    return list(set(vals))


def mapper(profile):
    results = {}
    for k in profile.keys():
        if k in fields_of_interest:
            results[k] = fields_of_interest[k][profile[k]]
    if profile['Age'] != '':
        age = int(profile['Age'])
        if age >= 18 and age < 25:
            results['Age'] = 'young'
        if age >= 25 and age < 40:
            results['Age'] = 'middle-aged'
        if age >= 40 and age < 60:
            results['Age'] = 'old'
        if age >= 60 and age < 100:
            results['Age'] = 'very old'
    else:
        results['Age'] = ''
    return results



def main():
    # Read data from file
    lines = open("../out_of_many/main/Study1_data/ppfull.csv", "r").readlines()
    dmap = {}
    header_vals = lines[0].strip().split(",")
    for line in lines[1:]:
        parts = line.strip().split(",")
        if len(parts) != len(header_vals):
            print("Error on line: " + line)
            continue
        dmap[parts[0]] = {}
        for hind, h in enumerate(header_vals):
            dmap[parts[0]][header_vals[hind]] = parts[hind]

    # Results dictionary
    results = []
    ids = dmap.keys()
    for id in tqdm(ids):
        try:
            user_profile = mapper(dmap[id])
            prompt = ""
            if user_profile['Ideo'] != '':
                prompt += "Ideologically, I describe myself as " + user_profile['Ideo'] + ". "
            if user_profile['PID7'] != '':
                prompt += "Politically, I " + user_profile['PID7'] + ". "
            if user_profile['WHITE'] == 'White':
                prompt += "Racially, I am white. "
            if user_profile['Hisp'] == 'Hispanic':
                prompt += "Racially, I am Hispanic. "
            if user_profile['Gender'] != '':
                prompt += "I am " + user_profile['Gender'] + ". "
            if user_profile['Inc'] != '':
                prompt += "Financially, I am " + user_profile['Inc'] + ". "
            if user_profile['Age'] != '':
                prompt += "In terms of my age, I am " + user_profile['Age'] + ". "
            user_profile.update({'prompt':prompt})
            results.append(user_profile)
        except:
            pass
    df = pd.DataFrame(results)
    df.to_csv("prompts.csv")
    return df
        
descs = main()


Error on line: R_5cf9ug7blHKzEDX,"$500K to $1,000K",Not Hispanic,Liberal,38,liberal,socially conscious,politically correct,kind,wealthy,conservative,closed minded,ignorant,Weak D,Democrat,White,4,4,Female,wealthy|conservative|closed minded|ignorant,liberal|socially conscious|politically correct|kind,0,0

Error on line: R_eb4thrlwqSPNGi9,"More than $1,000K",Hispanic,Moderate/Haven't thought about it,24,poor,,,,rich,,,,Strong R,Republicans,Non-white,1,1,Male,rich|||,poor|||,1,1

Error on line: R_ebxaYTTERHs3RJP,"More than $1,000K",Not Hispanic,Conservative,26,poor,black,lazy,uneducated,smart,powerful,values,educated,Strong R,Republicans,White,4,4,Female,smart|powerful|values|educated,poor|black|lazy|uneducated,0,0

Error on line: R_D7RQwYPEczU0vQt,$25K to $50K,Not Hispanic,Liberal,25,,"left wing, people minded, considerate, pro-choice",,,"stubborn, backwards, racist, rich",,,,Weak D,Democrat,White,1,1,Female,"stubborn, backwards, racist, rich|||","|left wing, people minded, considerate, 

100%|██████████| 2107/2107 [00:00<00:00, 157779.70it/s]


In [37]:
descs

,Inc,Hisp,Ideo,PID7,WHITE,Gender,Age,prompt
0,middle-class,,slightly conservative,am an independent,white,female,old,"Ideologically, I describe myself as slightly c..."
1,middle-class,,slightly conservative,am a weak Republican,white,male,old,"Ideologically, I describe myself as slightly c..."
2,very poor,,liberal,lean towards Democrats,white,female,old,"Ideologically, I describe myself as liberal. P..."
3,middle-class,,slightly conservative,lean towards Democrats,white,male,old,"Ideologically, I describe myself as slightly c..."
4,middle-class,,conservative,am a strong Republican,white,male,old,"Ideologically, I describe myself as conservati..."
...,...,...,...,...,...,...,...,...
1588,middle-class,,extremely liberal,am a strong Democrat,,female,,"Ideologically, I describe myself as extremely ..."
1589,poor,,moderate,am a weak Democrat,,female,young,"Ideologically, I describe myself as moderate. ..."
1590,,,,,,,,
1591,upper-class,,liberal,am a weak Democrat,,male,young,"Ideologically, I describe myself as liberal. P..."


In [11]:
dmap

{'R_dbTpgNCUaQtIifr': {'ID': 'R_dbTpgNCUaQtIifr',
  'Inc': '$100K to $150K',
  'Hisp': 'Not Hispanic',
  'Ideo': 'Slightly conservative',
  'Age': '40',
  'HD1': 'idiots',
  'HD2': '',
  'HD3': '',
  'HD4': '',
  'HR1': 'conservative',
  'HR2': '',
  'HR3': '',
  'HR4': '',
  'PID7': 'Ind',
  'PID3_p': 'Independent (excludes leaners)',
  'WHITE': 'White',
  'HDans': '1',
  'HRans': '1',
  'Gender': 'Female',
  'GenR': 'conservative|||',
  'GenD': 'idiots|||',
  'dkR': '1',
  'dkD': '1'},
 'R_51QVYKmeIc9iLpH': {'ID': 'R_51QVYKmeIc9iLpH',
  'Inc': '$100K to $150K',
  'Hisp': 'Not Hispanic',
  'Ideo': 'Slightly conservative',
  'Age': '50',
  'HD1': 'liberal',
  'HD2': '',
  'HD3': '',
  'HD4': '',
  'HR1': 'conservative',
  'HR2': '',
  'HR3': '',
  'HR4': '',
  'PID7': 'Weak R',
  'PID3_p': 'Republicans',
  'WHITE': 'White',
  'HDans': '1',
  'HRans': '1',
  'Gender': 'Male',
  'GenR': 'conservative|||',
  'GenD': 'liberal|||',
  'dkR': '1',
  'dkD': '1'},
 'R_9Fy2uWRvtBBSouh': {'ID': '